In [1]:
import configparser
import pymysql
import csv

In [2]:
def create_connection():
    config = configparser.ConfigParser()
    config.read('config.ini')
    host = config['mysql']['host']
    user = config['mysql']['user']
    password = config['mysql']['password']

    try:
        conn = pymysql.connect(
            host=host,
            user=user,
            password=password
        )
        print("Successful Connection")
        cursor = conn.cursor()
        return conn, cursor
    except pymysql.Error as e:
        print("Connection Error:", e)
        return None, None

In [3]:
def load_grammydb(cursor, csv_file):
    cursor.execute("CREATE DATABASE IF NOT EXISTS grammydb")
    cursor.execute("USE grammydb")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS grammy (
        year INT NOT NULL,
        title VARCHAR(255),
        published_at DATETIME DEFAULT CURRENT_TIMESTAMP,
        updated_at DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        category VARCHAR(255),
        nominee VARCHAR(255),
        artist VARCHAR(255),
        workers VARCHAR(1000),
        img TEXT,
        winner BOOLEAN
    ) """)

    with open(csv_file, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)
        for row in csv_reader:
            year, title, published_at, updated_at, category, nominee, artist, workers, img, winner = row
            winner = True if winner.lower() == 'true' else False
            sql = """
            INSERT INTO grammy
            (year, title, published_at, updated_at, category, nominee, artist, workers, img, winner)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (year, title, published_at, updated_at, category, nominee, artist, workers, img, winner))
        print("Data inserted successfully!!")


In [4]:
conn, cursor = create_connection()
load_grammydb(cursor, '../Datasets/the_grammy_awards.csv')
conn.commit()
conn.close()

Successful Connection
Data inserted successfully!!
